# Subword Tokenization and Byte-Pair Encoding

This assignment's goals are to be able to

1. Implement the Byte-Pair Encoding algorithm
2. Explain the trade-offs between character-level and word-level tokenization, and how subword tokenization methods like BPE form a compromise.
3. Recognize failure conditions in NLP systems related to tokenization choices. 

The n-gram models we've seen so far have a major flaw: all words (here, space separated sequences of characters!) are assumed to be entirely unrelated as far as the model is concerned! Recall our morphology lecture where I tried to (and hopefully did) convince you that if we want good representations of language (like the ones that help us predict the next word of a sentence) we'd like those representations to encode structural similarities between words.

Consider, for example, a hypothetical corpus that contains many instances of the phrase the bigram `("write", "code")` (from phrases like "I need to write code later") but 0 instances of `("rewrite", "code")`. What probability does our model assign the sentence "I need to rewrite code later"?

In [ ]:
# you can notate your answer here for your own notes if you'd like!

### Word-Level Warm-Up

Our n-gram models are currently what we call *word-level* models: models that have their atomic units as words in a vocabulary. While we like to spell-out our word tokens in our representations, our model treats different words in the vocabulary as unrelated, unstructured units --- elements in the set $V$. In fact, our language models would work just as well if every element in our vocabulary was replaced by a unique integer. In fact, this is often done in practice! Partially to compress our representations (now you just need an array rather than a dictionary, because your keys are now indicies!), and partially because this will become computationally convenient to us when we move into neural network techniques later in the course!

To get comfortable doing that, I'll ask you to warm-up by constructing a "word2idx" dictionary that maps a word to it's index in the vocabulary with a function `get_word2idx` (which, for now, will be a nice small example vocab!), as well as an `ngram2idxs` function that turns your n-gram tuples into integer tuples!

**Python tip of day**: if you want to iterate through an iterable and keep track of both values and indices, the *#pythonic* way to do so is to use `enumerate`, which the following code block demonstrates!

In [2]:
vocab = ["the", "dog", "cat", "is", "are", "dogs", "cats", "happy", "sleepy"]

In [3]:
# Bad! 
for i in range(len(vocab)):
    print(i, vocab[i])

# Do this instead!
for i, w in enumerate(vocab):
    print(i, w)

0 the
1 dog
2 cat
3 is
4 are
5 dogs
6 cats
7 happy
0 the
1 dog
2 cat
3 is
4 are
5 dogs
6 cats
7 happy


In [10]:
from typing import Mapping, Sequence, Iterable

# TODO: complete get_word2idx and ngram2idxs
def get_word2idx(vocab : Sequence[str]) -> Mapping[str, int]:
    return {}

def ngram2idxs(word2idx : Mapping[str, int], ngram : Sequence[str]) -> Sequence[int]:
    return ()

print(ngram2idxs(get_word2idx(vocab), ("dog", "is", "happy")))

()


Now, a hypothetical to consider: Why don't we assign each n-gram to a number to have even more compressed representations! That is, instead of representing `("dog", "is", "happy")` with `(1, 3, 7)`, why don't we assign it a single number, say, `10`?

There are a couple of reasonable answers, but consider how having a structured representation like `(1, 3, 7)` encodes the relationship between different n-grams. Would it be straightforward to implement backoff if we had a vocabulary of n-grams?

### Character-Level Models

Now, to an alternative. These n-grams are *word*-level models. What if we fully-committed to subword information text provides (and abandon the nonsense of space-separated words) and build a *character*-level n-gram model. That is, our vocabulary contains every (unicode) character (including spaces!) and we model with characters as our atomic units!

If we assume this, tokenization is easy... except for a couple of technical notes. Like our words, to emphasize that there is nothing about the characters themselves (other than their identity) that our models can consider, we will subsititute an integer for the character itself. Since we're working with characters, there are standard integer encodings for characters (in Python 3, [Unicode](https://home.unicode.org/)). the `ord` function maps characters to their unicode values.

**Python tip of the day #2**: As you might guess, doing this will make the number of tokens in a corpus very large. Of course, since a string itself is a sequence of characters (in Python, just strings of length 1) this doesn't actually take up much space. However, this simplicity makes it a good place to introduce python [generators](https://docs.python.org/3/glossary.html#term-generator), a special kind of function that allows you to create something iterable (comparable to returning a list you want to for-loop through) but without having to take up the memory required to store every element of that list at the same time. You do this by *generating* the next value as you need it. Because we generate elements one-by-one, these are *iterable* (remember the iterator interface in Java from Data Structures!) but not indexible (you can't access the value at a specific point!). 

Take a look at the example below!

In [18]:
data = "The dog is sleepy"

def char_tokenize(corpus : str) -> Iterable[str]:
    for char in data:
        yield ord(char)

def char_tokenize_no_generator(corpus : str) -> Iterable[str]:
    out = []
    for char in data:
        out.append(ord(char))
    return out

print("--- with generator")
for token in char_tokenize(data):
    print(token)
    
print("--- without generator")
for token in char_tokenize(data):
    print(token)

--- with generator
84
104
101
32
100
111
103
32
105
115
32
115
108
101
101
112
121
--- without generator
84
104
101
32
100
111
103
32
105
115
32
115
108
101
101
112
121


Now there's a bit of an issue with character-level models:

Say I train a 5-gram model on words and a 5-gram model on characters. **Which model do you think will form more coherent outputs if you generated from them? How does $n$ need to change to get comparable levels of contextual awareness?**

On the other hand, **do we have to worry about out-of-vocabulary tokens in a character-level model?** Trade-offs!

# Byte-Pair Encoding (BPE)



In comes Byte-Pair Encoding, an algorithm from data compression that we repurpose to achieve one goal: Let's make tokens that are smaller than space-separated words, but larger than individual Unicode characters. How does it work: by finding frequent *pairs* of tokens and merging them together into a new, larger token in our vocabulary! We do this until we get our desired number of tokens in our vocabulary, which acts as a parameter to the algorithm. 

We can then reproduce this sequence of merges on any corpus in order to *tokenize* that text! This is a way to *learn* good subword token representations from a corpus, using the heuristic of "pairs that appear together often are likely morphemes (i.e., meaningful units!)." 

Technical note: As the name implies, we typically do this over bytes instead of characters. To make this conversion simple, we will limit ourselves to the ASCII text encoding, which represents each character with a byte. Conveniently, the first 256 unicode characters are exactly the 256 ASCII characters in the same order! 

For the rest of this activity, we'll implement the BPE algorithm to build a tokenizer from scratch. It's worth noting that this algorithm is used by big fancy LLMs (i.e., the GPTs that we have some public knowledge about use a BPE variant!). 

In [101]:
class BPETokenizer:
    def __init__(self, train_data : str, vocab_size : int, verbose=False):
        assert vocab_size > 257 # Our vocab must at least contain each possible byte value as our alphabet, plus the end-of-work byte 
        self.EOW = 256
        self.vocab = [chr(i) for i in range(256)] + ["<EOW>"]
        self.train_data = self.preprocess(train_data)
        self.merges = [] # formatted as (token1, token2, merged_token)

        self.train(vocab_size - 257, verbose=verbose)
        
    def preprocess(self, train_data : str) -> Sequence[Sequence[int]]:
        """ Convert a string into a integer sequence AND split by word
            to ensure no cross-word merges. 
        """
        return [([ord(l) for l in w] + [self.EOW]) 
                for w in train_data.split()]
    
    def train(self, num_merges : int, verbose=False):
        for i in range(num_merges):
            # TODO: Get the most frequent bigram
            max_bigram = ()
            
            # TODO: Create a new vocab item for the bigram
            new_idx = None

            # Print a message for each merge if V
            if verbose: print("Merge {}: {}, {} -> {} ({})".format(
                i, max_bigram, new_idx, self.vocab[new_idx]))
            
            # TODO: Create and store a new merge rule
            new_merge_rule = ()

            self.train_data = self.merge(new_merge_rule, self.train_data)
            
    def get_counts(self) -> Mapping[Sequence[int], int]:
        # TODO get bigram counts given current tokenization
        return {}

    def merge(self, merge_rule : Sequence[int], data : Sequence[int]) -> Sequence[int]:
        return []

    def encode(self, data : str) -> Sequence[int]:
        data = self.preprocess(data)
        for merge_rule in self.merges:
            data = self.merge(merge_rule, data)
        return [i for w in data for i in w]

    def decode(self, data : Sequence[int]) -> str:
        return "".join((self.vocab[i] for i in data))
        
    def decode_with_splits(self, data : Sequence[int]) -> str:
        return "_".join((self.vocab[i] for i in data))

Here is the implementation order I recommend:

1. You have `__init__`, `preprocess`, `decode` and `decode_with_splits` given to you. Get an idea of how the given code is structured, though for activities feel free to modify the given structure as you see fit.
2. Look at the provided method signatures for `get_counts` (which counts the number of adjacent pairs in the training data) and `merge`(which replaces pairs of adjacent tokens with a new, merged token).
3. Write `train` referencing the pseudocode from the reading and provided comments. 
4. Implement `get_counts` (referencing your n-gram counting code from before! it'll be similar!)
5. Implement `merge`
6. Test to make sure `train`, `get_counts`, and `merge` work correctly.
7. Implement `encode`. 

In [1]:
# Mimic the example counts from J&M
test_string = "low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new"
tokenizer = BPETokenizer(test_string, 257 + 4, verbose=True)

encoded = tokenizer.encode("newfoundlander")
print(encoded)
print(tokenizer.decode(encoded))

# See the subword pieces the model chose!
print(tokenizer.decode_with_splits(encoded))

NameError: name 'BPETokenizer' is not defined

If all goes well, this should match the example from the textbook, with merges, in order, "er", "er<EOW>", "ne", and "new". If you up the final requested vocab size, you should get further merges you can check against the reading!

**Bonus: Training on a real corpus!**

Given time, you can train your BPE tokenizer with various sizes on a "real" corpus, like (our classic!) Austen's Emma!

This may take a little bit to run: your implementation is meant to be readable, not fast (and I don't expect you to do any optimizations!), but it's worth noting that as given the algorithm is $O(kn)$ where $k$ is the number of merges/vocab size and $n$ is the length of the data in bytes! 

If you'd like to look at a larger corpus (and have some spare time!), you can run this code on the Brown Corpus or even something larger. This is a real, working BPE implementation! 

It may be helpful to notice what kinds of tokens are formed by this process. Do all of them seem morphologically legitimate? What kinds of errors can you imagine a model would make if these were it's atomic units?

In [106]:
with open("austen-emma.txt") as emma_f:
    emma = emma_f.read()

EmmaTokenizer = BPETokenizer(emma, 557)

# Play around with tokenization here! 

['\x00', '\x01', '\x02', '\x03', '\x04', '\x05', '\x06', '\x07', '\x08', '\t', '\n', '\x0b', '\x0c', '\r', '\x0e', '\x0f', '\x10', '\x11', '\x12', '\x13', '\x14', '\x15', '\x16', '\x17', '\x18', '\x19', '\x1a', '\x1b', '\x1c', '\x1d', '\x1e', '\x1f', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x81', '\x82', '\x83', '\x84', '\x85', '\x86', '\x87', '\x88', '\x89', '\x8a', '\x8b', '\x8c', '\x8d', '\x8e', '\x8f', '\x90', '\x91', '\x92', '\x93', '\x94', '\x95', '\x96', '\x97', '\x98', '\x99', '\x9a', '\x9b', '\x9c', '\x9d', '\x9e', '\x9f', '\xa0', '

**Bonus 2: Working with a "real" tokenizer. **

Now that you understand how BPE works, it might be worth messing around a bit with a "real" BPE tokenizer --- that of GPT2, the predecessor the GPTs that power ChatGPT and it's ilk! 

Below is a snippet of code that will let you tokenize things using GPT-2's tokenizer to see how words are actually decomposed. Note that the strange symbol that prefaces most words is the equivalent of our `<EOW>` token, except it marks the beginning and not the end of words (what is, in some sense, an arbitrary choice!). 

This code does use a library I haven't asked you to install yet, so if you'd like to test this out, you can install the (HuggingFace (HF) Transformers library)[https://huggingface.co/docs/transformers/en/index] which has an implementation of the GPT-2 tokenizer with a pretrained model available. The next cell prompts you to install it, and afterward you may need to restart your kernel to import transformers.

You could also move to something more fancy, like GPT3/GPT4/etc. tokenizers that you can test using (OpenAI's tokenization demo)[https://platform.openai.com/tokenizer]. These aren't running locally like they would be with HF's implementations, but they will suffice for your exploration!

In [108]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


While you explore, I recommend looking at...

1. Multi-digit numbers (Would an LM trained on these have a great representation of numbers?)
2. Morphologically complex words (Are these linguistically sensible?)
3. [" SolidGoldMagikarp"](https://arxiv.org/abs/2405.05417)

In [116]:
from transformers import AutoTokenizer

# Load the GPT2 Tokenizer
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

# Change the example sentence!
example = "This is an example to see when words are decomposed into subword pieces"
tokenizer.tokenize(example)

['This',
 'Ġis',
 'Ġan',
 'Ġexample',
 'Ġto',
 'Ġsee',
 'Ġwhen',
 'Ġwords',
 'Ġare',
 'Ġdecomp',
 'osed',
 'Ġinto',
 'Ġsub',
 'word',
 'Ġpieces']